## Generating Sequences to Match an Target $C_0$

What we have is an ML model g : $x \rightarrow C_0$ and a CE model f : $x \rightarrow C_0$, where $x \in \set{0, 1}^{200}$ is a one-hot encoded vector representing a DNA sequence under the following encoding: <br>
A -> [1, 0, 0, 0] <br>
T -> [0, 1, 0, 0] <br>
C -> [0, 0, 1, 0] <br>
G -> [0, 0, 0, 1]

Given a target $C_0$, we want to build an algorithm to solve for $x'$, a sequence such that $f(x') \approx C_0$ <Br>
<br>
The algorithm should also handle:
1. Hard enforcement of specific motifs (hard = at a specific position)
2. Soft enforcement of specific motifs (soft = somewhere in the sequence)
3. Enforcement of multiple Hard and Soft motifs

#### Chosen algorithm: `Monte Carlo Simulated Annealing`

In [93]:
from dataclasses import dataclass
proj_dir = '/Users/prabh/200 RESEARCH/Cluster Expansion Project'

# gives the notebook access to the cluster_expansion personal library
from sys import path
path.append(proj_dir)
import cluster_expansion as ce
from cluster_expansion import os, pd, np, plt, tf, Markdown, display

import random

In [94]:
BASES = ['A', 'T', 'C', 'G']
SEQ_LEN = 50

# Load consolidated interaction tensor
G_PATH = ("../InteractionTensors/100M Sequences/Symmetric/GC_100M.npy")
GC = np.load(G_PATH)

# Load ML model
MODEL_PATH = ("../CNN_models/RC_invariant/Symmetric/S_C0freeAvg.keras")
S = tf.keras.models.load_model(MODEL_PATH)

In [95]:
@dataclass
class HardMotif:
  motif: str
  start_idx: int

In [103]:
# Enforce the presence of a motif at a specific sequence
def LockedPositions(hard_motifs: list[HardMotif]) -> set[int]:
    lock = set()
    for hm in hard_motifs:
        for i in range(len(hm.motif)):
            lock.add(hm.start_idx + i)
    return lock

def InitSeq(L: int, hard_motifs: list[HardMotif]) -> tuple[str, set[int]]:
    seq_list = [random.choice(BASES) for _ in range(L)]
    lock = set()
    for hm in hard_motifs:
        m = hm.motif.upper()
        s = hm.start_idx
        for i, ch in enumerate(m):
            seq_list[s + i] = ch
            lock.add(s + i)
    return "".join(seq_list), lock

def SoftMotifLoss(seq: str, soft_motifs: list[SoftMotif], penalty: float = 1.0) -> float:
    return sum(0.0 if soft_motif.motif.upper() in seq else penalty for soft_motif in soft_motifs)

def NetLoss(seq: str, C0: float, targetC0: float):
    return (C0 - targetC0) ** 2

In [104]:
# Predict a sequence's C0 using ML model
def S_pred(model, seq: str) -> float:
  oh = ce.Data2Onehot([seq], verbose = False).reshape(-1, 200 , 1)
  c0pred = model.predict(oh, verbose = 0)[0,0]
  return c0pred

# Predict a sequence's C0 using G tensor
def G_pred(GC_tensor, seq: str) -> float:
  avgC0 = -0.18
  oh = ce.Data2Onehot([seq], verbose=False).reshape(-1)
  c0pred = ce.predictC0(onehotSeq=oh, avgC0=avgC0, G = GC_tensor)
  return c0pred

In [105]:
# Mutate up to three bases at a time (makes sense as the interaction tensor captures up to triplet interactions)
def KMutation(seq: str, lock: set[int], k: int = 3) -> str:
    """
    Propose a new sequence by mutating k distinct unlocked positions.
    Returns a NEW sequence string.
    """
    L = len(seq)
    seq_list = list(seq)    # Makes string mutable

    unlocked = [i for i in range(L) if i not in lock]
    if len(unlocked) == 0:
        return seq          # Positions unavailable for mutation

    # If k > number of unlocked positions, truncate k
    k_eff = min(k, len(unlocked))
    positions = random.sample(unlocked, k_eff)

    # perform mutations on sampled positions
    for pos in positions:
        old = seq_list[pos]
        seq_list[pos] = random.choice([b for b in BASES if b != old])

    # join back to a string
    return "".join(seq_list)


In [106]:
def MCSA(
  targetC0: float,
  GC_tensor,
  hard_motifs: list[HardMotif],
  avgC0: float = -0.18,
  k_mut: int = 3,
  n_steps: int = 10_000,
  T_start: float = 1.0,
  T_end: float = 0.01,
  tolerance: float = 1e-2,
  verbose: bool = False
) -> (str, float):

  # Seed random stuff
  seed = random.randrange(2**31 -1)
  random.seed(seed)
  np.random.seed(seed)

  # Initialize sequence and c0
  curr_seq, lockedPos = InitSeq(L = SEQ_LEN, hard_motifs = hard_motifs)
  curr_c0 = G_pred(GC_tensor=GC_tensor, seq=curr_seq)
  curr_E = NetLoss(seq=curr_seq, C0=curr_c0, targetC0=targetC0)

  # Initialize best sequences
  best_seq = curr_seq
  best_c0 = curr_c0
  best_E = curr_E

  # Iterate
  for t in range(1, n_steps+1):

    # 'Temperature' controls - exponentially decays
    a = (t - 1) / (n_steps - 1)
    T = T_start * (T_end / T_start) ** a

    # Mutate
    seq_candidate = KMutation(seq=curr_seq, lock=lockedPos, k=k_mut)
    c0_candidate = G_pred(GC_tensor=GC_tensor, seq=seq_candidate)
    E_candidate = NetLoss(seq=seq_candidate, C0=c0_candidate, targetC0=targetC0)

    dE = E_candidate - curr_E

    # Accept/Reject mutation
      # random number < e^{-dE/T}, but if T < 1e-12 (rounding errors), it maxes at 1e-12
    if (dE <= 0) or (random.random() < np.exp(-dE / max(T, 1e-12))):
      curr_seq = seq_candidate
      curr_c0 = c0_candidate
      curr_E = E_candidate

      if curr_E < best_E:
        best_seq = curr_seq
        best_c0 = curr_c0
        best_E = curr_E

    if verbose and (t % (n_steps // 10) == 0):
      print(f"step={t} T={T:.4g} curr_c0={curr_c0:+.4f} best_c0={best_c0:+.4f} best_E={best_E:.4g}")
    
    if abs(best_c0 - targetC0) <= tolerance:
      break

  return best_seq, best_c0


In [ ]:
hard_motifs = [HardMotif('ATCG', 10)]  # or [HardMotif("ACGT", 10)]

best_seq, best_c0 = MCSA(
    targetC0=0,
    GC_tensor=GC,
    hard_motifs=hard_motifs,
    k_mut=3,
    n_steps=40_000,      # small first run
    T_start=1.0,
    T_end=0.01,
    tolerance=1e-4,
    verbose=True
)

best_c0_ml = S_pred(S, best_seq)

print(f"Best sequence ({len(best_seq)} nt):\n{best_seq}\n")
print(f"GC predicted C0: {float(best_c0):+.6f}")
print(f"ML predicted C0: {float(best_c0_ml):+.6f}")


Best sequence (50 nt):
AGCGCACTGTATCGCTAATATAAAGACTATCAGGCTACGGTCTGTGGAAC

GC predicted C0: -0.000091
ML predicted C0: -0.061924


In [117]:
motif = 'ATCG'
for i in range(50 - len(motif)):
  hard_motifs = [HardMotif('ATCG', i)]

  best_seq, best_c0 = MCSA(
      targetC0=1,
      GC_tensor=GC,
      hard_motifs=hard_motifs,
      k_mut=3,
      n_steps=1_000,      # small first run
      T_start=1.0,
      T_end=0.01,
      tolerance=1e-4,
      verbose=False
  )

  best_c0_ml = S_pred(S, best_seq)

  print(f"Best sequence ({len(best_seq)} nt): {best_seq}\t{best_c0:+.3f}\t{best_c0_ml:+.3f}")
  # print(f"GC predicted C0: {float(best_c0):+.6f}")
  # print(f"ML predicted C0: {float(best_c0_ml):+.6f}")


Best sequence (50 nt): ATCGCCCTGAAACTGGGTAAAAAGGCCTGAACAGGACTATGGTTATACTA	+1.000	+0.924
Best sequence (50 nt): TATCGAAAATGGGCCCCCGTAAGCTCTCGTAAGGGCCGTTTGTCAGGCGA	+1.001	+0.955
Best sequence (50 nt): AGATCGGAAAACATCCATCAGAGGCGCATAAAATGAAGCGAAAACGCATA	+0.997	+1.170
Best sequence (50 nt): AGTATCGGGGGATCTCAATACGGGCTCCTATAGGGTGTCCGGGACGCCTT	+1.001	+0.650
Best sequence (50 nt): AGAAATCGGGTTTCTAGCATCCCCTGGGGGGTCCTTCTGTTCGTTTATAC	+1.000	+1.108
Best sequence (50 nt): GTGGTATCGTTATAGCCCAAATCATGCTTAGAAAGGCCTTCTCCCTACTT	+0.999	+1.001
Best sequence (50 nt): ACGCATATCGGGCGAGAAATTACGGGAATTATAAGCCTTAACAGGCATAC	+1.003	+0.875
Best sequence (50 nt): TACTCTCATCGGGGTCTAAGAAGGGGCCGGAGAGGCCGCTGCCAGCGTTT	+0.999	+0.750
Best sequence (50 nt): ACCAAAAGATCGCCCGCAAACGCCCTTAAGCCTATGCAGCAGAGCTTTCT	+1.001	+0.825
Best sequence (50 nt): AACCCGGTAATCGCCTTCGTGGAGTCGTATTTAGGGATATTTCTGCGTGC	+1.002	+0.978
Best sequence (50 nt): ATTTAAGCGTATCGAGCAGCAAGATTGGGCTTCTGGGGCTCGCAGGGGGG	+1.000	+0.867
Best sequence (50 nt): AGAGGAATT